### Imports and function definitions

In [72]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

In [73]:
# Run this if you want to build in 3.0 rather than 2.0
from config import Settings
Settings.uvcustommodulelocation='C:\\Users\\msubbarao\\SCISS\\Uniview Theater 3.0\\Custom Modules'

### USES Conf Templates

In [74]:
Template='''mesh
{
    data spacecrafts ./modules/$folderName/LISA.uvstream
    dataHints spacecrafts disablePostprocessing

    # use a radius to avoid frustum culling
    #cullRadius $cr
    glslVersion 420 

    propertyCollection
    {
        __objectName__
        {
            vec3f laserColor 1.0 0.2 0.2 | public | desc "laser color" | widget color_rgb
            vec1f phaseShift 0.5 | public | desc "phase wrt Earth" | widget slider | range 0 1
        }
    }

    pass
    {
        passEnable true
        useDataObject spacecrafts

        shader 
        {
            type defaultMeshShader
            {
                vertexShader ./modules/$folderName/LISA.vs
                fragmentShader ./modules/$folderName/LISA.fs
                geometryShader ./modules/$folderName/LISA.gs

                #stateManagerVar __objectName__.Alpha alpha
                stateManagerVar __objectName__.laserColor laserColor
                stateManagerVar __objectName__.phaseShift phaseShift
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_DEPTH_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_LINE_WIDTH 3
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE
                }
            }
        }
    }    
}
'''

### uvSTREAM

In [75]:
UVSTREAM='''<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<DataStream version="0.1">
    <PrimitiveType>lines_adjacency</PrimitiveType>
    <DataBindings>
        <Variable>
            <Name>sc1</Name>         
            <Type>vec3f</Type>
        </Variable>
        <Variable>
            <Name>sc2</Name>         
            <Type>vec3f</Type>
        </Variable>             
        <Variable>
            <Name>sc3</Name>         
            <Type>vec3f</Type>
        </Variable>             
        <Variable>
            <Name>time</Name>         
            <Type>float</Type>
        </Variable>                       
    </DataBindings>

    <CSVData>
        <Path>spacecrafts.csv</Path>
        <Headers>
            <Variable>              
                <Cols>0,1,2</Cols>
            </Variable>
            <Variable>              
                <Cols>3,4,5</Cols>
            </Variable>                     
            <Variable>              
                <Cols>6,7,8</Cols>
            </Variable>                     
            <Variable>              
                <Cols>9</Cols>
            </Variable>                                      
        </Headers>
    </CSVData>
</DataStream>  
'''

In [76]:
uvStreamFile=open('data/LISA.uvstream','w')
uvStreamFile.write(UVSTREAM)
uvStreamFile.close()

## LISA Class

In [77]:
class LISA():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("LISA.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("LISA.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("LISA.gs",dataFolder)
        self.Scale = 1
        self.cr = 1.0e6
    def setScale(self,Scale):
        self.Scale = Scale
    def generatemod(self):
        self.object.setgeometry("LISA.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = "LISA.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [78]:
model = LISA(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
parentScene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [79]:
scene.setname("LISA")

model.object.setname("LISA")
model.object.setcoord("SolarSystem")
model.object.setguiname("/KavliLecture/Larson/LISA orbits")
model.object.settargetradius(50000)
model.object.setscalefactor(1.0)
model.object.showatstartup(False)

modinfo.setname("LISA orbits")
modinfo.setauthor("Shane Larson<sup>1</sup> and Mark SubbaRao<sup>1</sup><br />(1)Northwestern University,<br />(2)Adler Planetarium")
modinfo.cleardependencies()
modinfo.setdesc("potential LISA orbits<br />")
modinfo.setthumbnail("data/thumbnail.png")
modinfo.setversion("1.0")

generator.generate("LISA",[],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

Unable to connect to Uniview


### Helper Functions for modifing code
##### Reload Module and Shaders in Uniview

In [20]:
uvmod.Utility.senduvcommand(model.object.name+".reload")

Copy modified Shader files and reload

In [21]:
from config import Settings

In [39]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

In [71]:
scene.name

'LISA'